In [1]:
import torch
import random
import numpy as np
from pathlib import Path
from sklearn.preprocessing import LabelEncoder

In [2]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('Training on CPU')
    DEVICE = torch.device("cpu")
else:
    print('Training on GPU')
    DEVICE = torch.device("cuda:0")

SEED = 666
seed_everything(SEED)

TRAIN_DIR = Path('./imagewoof2-320/train')
TEST_DIR = Path('./imagewoof2-320/val')

train_val_files = sorted(list(TRAIN_DIR.rglob('*.JPEG')))
test_files = sorted(list(TEST_DIR.rglob('*.JPEG')))

train_val_labels = [path.parent.name for path in train_val_files]
LABEL_ENCODER = LabelEncoder()
LABEL_ENCODER.fit(train_val_labels)


Training on CPU


LabelEncoder()

In [3]:
from datasets import Imagewoof
from models import resnet18


In [4]:
train_dataset = Imagewoof(train_val_files, LABEL_ENCODER)
test_dataset = Imagewoof(test_files, LABEL_ENCODER)


In [5]:
resnet = resnet18(10, DEVICE)

In [9]:
from teacher_train import train, predict
train(train_dataset, test_dataset, resnet, 5, 64, DEVICE)

epoch:   0%|          | 0/5 [00:04<?, ?it/s]


RuntimeError: size mismatch, m1: [64 x 2048], m2: [512 x 2048] at /pytorch/aten/src/TH/generic/THTensorMath.cpp:41

In [8]:
from torch.utils.data import DataLoader
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)
probas = predict(resnet, test_loader, temp=10)
probas

NameError: name 'predict' is not defined